In [62]:
import pandas as pd
from sklearn.ensemble import AdaBoostRegressor
from sklearn.preprocessing import scale
from sklearn.model_selection import GridSearchCV,cross_val_score,train_test_split
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.decomposition import PCA
import numpy as np
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [2]:
features=pd.read_csv('features.csv')

In [3]:
features.head()

,Unnamed: 0,pickup_datetime,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,travel_time,date_of_year,year,mo,...,day_of_week,weekday,day_binned,day_hour,time_binned,day_number,day_cosine,day_sine,time_cosine,time_sine
0,0,2016-01-18 03:25:25,40.731525,-73.988670,40.760036,-73.984856,626,2016-01-18,2016,1,...,1.0,1.0,0.142857,3.0,0.125000,0.160714,0.532032,0.846724,0.707107,0.707107
1,1,2016-01-25 17:09:52,40.713608,-74.013718,40.765598,-73.980713,1192,2016-01-25,2016,1,...,1.0,1.0,0.142857,17.0,0.708333,0.244048,0.037391,0.999301,-0.258819,-0.965926
2,2,2016-01-05 00:17:41,40.773960,-73.874435,40.766693,-73.955414,842,2016-01-05,2016,1,...,2.0,1.0,0.285714,0.0,0.000000,0.285714,-0.222521,0.974928,1.000000,0.000000
3,3,2016-01-01 07:44:33,40.749718,-73.991570,40.768169,-73.912483,1054,2016-01-01,2016,1,...,5.0,1.0,0.714286,7.0,0.291667,0.755952,0.037391,-0.999301,-0.258819,0.965926
4,4,2016-05-29 14:34:21,40.762730,-73.974174,40.779640,-73.961823,538,2016-05-29,2016,5,...,0.0,0.0,0.000000,14.0,0.583333,0.083333,0.866025,0.500000,-0.866025,-0.500000


In [4]:
features.columns

Index(['Unnamed: 0', 'pickup_datetime', 'pickup_latitude', 'pickup_longitude',
       'dropoff_latitude', 'dropoff_longitude', 'travel_time', 'date_of_year',
       'year', 'mo', 'da', 'temp', 'visib', 'wdsp', 'gust', 'max', 'min',
       'prcp', 'sndp', 'fog', 'rain_drizzle', 'snow_ice_pellets', 'hail',
       'thunder', 'distance_in_km', 'Pickup Geohash', 'Dropoff Geohash',
       'day_of_week', 'weekday', 'day_binned', 'day_hour', 'time_binned',
       'day_number', 'day_cosine', 'day_sine', 'time_cosine', 'time_sine'],
      dtype='object')

### Correlation before prediction

In [5]:
features.corr()['travel_time']

Unnamed: 0           0.009183
pickup_latitude     -0.061481
pickup_longitude     0.096247
dropoff_latitude    -0.127299
dropoff_longitude    0.080626
travel_time          1.000000
year                      NaN
mo                   0.062146
da                   0.011005
temp                 0.057647
visib                0.009210
wdsp                 0.014894
gust                 0.032461
max                  0.052568
min                  0.056567
prcp                -0.010622
sndp                 0.026657
fog                 -0.017600
rain_drizzle        -0.002133
snow_ice_pellets    -0.031429
hail                      NaN
thunder                   NaN
distance_in_km       0.734535
day_of_week          0.028139
weekday              0.072720
day_binned           0.028139
day_hour             0.030524
time_binned          0.030524
day_number           0.031871
day_cosine          -0.069807
day_sine            -0.038148
time_cosine         -0.070464
time_sine           -0.040119
Name: trav

### Create variables for prediction

In [6]:
features_list=['pickup_latitude','pickup_longitude','dropoff_latitude','dropoff_longitude','distance_in_km','temp','wdsp','prcp','sndp','day_number','day_cosine','day_sine','time_cosine','time_sine']
y=features['travel_time']
X=features[features_list]

Train/Test split using KFold cross validation

In [7]:
kf = KFold(n_splits=3)
kf.get_n_splits(X)
print(kf)  
KFold(n_splits=3, random_state=2, shuffle=False)
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]

KFold(n_splits=3, random_state=None, shuffle=False)
TRAIN: [ 947508  947509  947510 ... 2842521 2842522 2842523] TEST: [     0      1      2 ... 947505 947506 947507]
TRAIN: [      0       1       2 ... 2842521 2842522 2842523] TEST: [ 947508  947509  947510 ... 1895013 1895014 1895015]
TRAIN: [      0       1       2 ... 1895013 1895014 1895015] TEST: [1895016 1895017 1895018 ... 2842521 2842522 2842523]


In [8]:
Xtrain=X_train[:100000]
ytrain=y_train[:100000]

In [9]:
Xtest=X_test[:25000]
ytest=y_test[:25000]

In [10]:
Xtrain=scale(Xtrain)
Xtest=scale(Xtest)

In [11]:
Xtrain.shape

(100000, 14)

In [12]:
Xtest.shape

(25000, 14)

### Function for prediction and score

In [77]:
def pred_score(clf,Xtrain,ytrain,Xtest,ytest):
    clf.fit(Xtrain,ytrain)
    y_pred_train=clf.predict(Xtrain)
    y_pred_test=clf.predict(Xtest)
    print('r2 score train :',r2_score(ytrain,y_pred_train))
    print('r2 score test :',r2_score(ytest,y_pred_test))

### AdaBoost regressor

Fine tune the regressor hyperparameters using cross validation

In [31]:
param_dist = {"learning_rate": [0.001,0.01,0.1],
              "n_estimators": [5,20,50]
              }
clf=AdaBoostRegressor()

# Instantiate the RandomizedSearchCV object: tree_cv
clf_cv = GridSearchCV(clf,param_dist, cv= 4 ,verbose=3)

# Fit it to the data
clf_cv.fit(Xtrain,ytrain)

# Print the tuned parameters and score
print("Tuned Decision Tree Parameters: {}".format(clf_cv.best_params_))
print("Best score is {}".format(clf_cv.best_score_))

Fitting 4 folds for each of 9 candidates, totalling 36 fits
[CV] learning_rate=0.001, n_estimators=5 .............................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  learning_rate=0.001, n_estimators=5, score=0.5541155954060089, total=   1.4s
[CV] learning_rate=0.001, n_estimators=5 .............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.4s remaining:    0.0s


[CV]  learning_rate=0.001, n_estimators=5, score=0.566403539367393, total=   1.3s
[CV] learning_rate=0.001, n_estimators=5 .............................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    2.7s remaining:    0.0s


[CV]  learning_rate=0.001, n_estimators=5, score=0.5602452402237024, total=   1.3s
[CV] learning_rate=0.001, n_estimators=5 .............................
[CV]  learning_rate=0.001, n_estimators=5, score=0.5558017546084937, total=   1.3s
[CV] learning_rate=0.001, n_estimators=20 ............................
[CV]  learning_rate=0.001, n_estimators=20, score=0.5553715468360592, total=   4.9s
[CV] learning_rate=0.001, n_estimators=20 ............................
[CV]  learning_rate=0.001, n_estimators=20, score=0.566271386947382, total=   5.0s
[CV] learning_rate=0.001, n_estimators=20 ............................
[CV]  learning_rate=0.001, n_estimators=20, score=0.5623802144792729, total=   4.8s
[CV] learning_rate=0.001, n_estimators=20 ............................
[CV]  learning_rate=0.001, n_estimators=20, score=0.5554103369827741, total=   4.9s
[CV] learning_rate=0.001, n_estimators=50 ............................
[CV]  learning_rate=0.001, n_estimators=50, score=0.5552116078175872, tot

[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed:  4.3min finished


Tuned Decision Tree Parameters: {'learning_rate': 0.1, 'n_estimators': 20}
Best score is 0.5706822068507567


In [32]:
y_pred_train=clf_cv.predict(Xtrain)
y_pred_test=clf_cv.predict(Xtest)

In [33]:
r2_score(ytrain,y_pred_train)

0.5669709644672545

In [34]:
r2_score(ytest,y_pred_test)

0.49369642404872205

### New training and feature importance

In [64]:
clf=AdaBoostRegressor(learning_rate=0.1,n_estimators=20)

In [65]:
clf.fit(Xtrain,ytrain)

AdaBoostRegressor(base_estimator=None, learning_rate=0.1, loss='linear',
         n_estimators=20, random_state=None)

In [66]:
len(clf.feature_importances_)

14

In [67]:
for i in range(len(clf.feature_importances_)):
    print(features_list[i],clf.feature_importances_[i])

pickup_latitude 0.0
pickup_longitude 0.0025698966723955324
dropoff_latitude 0.001026338814776694
dropoff_longitude 0.0
distance_in_km 0.9457300203977873
temp 0.0
wdsp 0.0
prcp 0.0
sndp 0.026958012887041926
day_number 0.0
day_cosine 0.0
day_sine 0.0
time_cosine 0.02371573122799847
time_sine 0.0


### PCA feature

In [68]:
def pca_dataset(X):
    clf=PCA(n_components=2)
    clf.fit(X)
    pca_x=clf.transform(X)
    pca_data=np.concatenate((X,pca_x),axis=1)
    return pca_data

In [54]:
pca_train=pca_dataset(Xtrain)
pca_test=pca_dataset(Xtest)

In [71]:
clf=AdaBoostRegressor(learning_rate=0.1,n_estimators=20)
pred_score(clf,pca_train,ytrain,pca_test,ytest)

r2 score train : 0.5708410408290184
r2 score test: 0.49719602370397775


We see small improvement with the PCA feature

### Default XGBoost

In [72]:
xgb=XGBRegressor()
pred_score(xgb,Xtrain,ytrain,Xtest,ytest)

/Users/camilleruppli/miniconda3/envs/py37/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/camilleruppli/miniconda3/envs/py37/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


r2 score train : 0.6834434135926122
r2 score test: 0.523432577993215


In [73]:
# see how pca impact default XGBoost prediction
xgb=XGBRegressor()
pred_score(xgb,pca_train,ytrain,pca_test,ytest)

/Users/camilleruppli/miniconda3/envs/py37/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/camilleruppli/miniconda3/envs/py37/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


r2 score train : 0.684249942325111
r2 score test: 0.489041630581849


We see improvement on performances compared to AdaBoost but performances are not convincing yet. We need parameter tuning to improve results here which can be found in the XGBoost notebook.
We see that PCA feature does not improve performances on the test set so we might not consider it in the future.

### Default Gradient Boosting

In [74]:
clf=GradientBoostingRegressor()
pred_score(clf,Xtrain,ytrain,Xtest,ytest)

r2 score train : 0.6837703241541504
r2 score test: 0.5201954688294059


In [76]:
clf=GradientBoostingRegressor()
pred_score(clf,pca_train,ytrain,pca_test,ytest)

r2 score train : 0.6848157129321398
r2 score test: 0.4810471605600539


We don't see significant improvement between gradient boosting and xgboost. As xgboost is supposed to be more efficient in term of computational speed we will stick with that model. PCA feature does not improve performances either